In [12]:
import pickle
import os

#get output results GC-MC
base_path = "../outputsGCMC/MovieLens100K" 


#Read the data
with open(os.path.join(base_path, "test_labels.pickle"), "rb") as file:
    test_labels = pickle.load(file,  encoding="latin1")
file.close()

with open(os.path.join(base_path, "class_values.pickle"), "rb") as file:
    class_values = pickle.load(file, encoding="latin1")
file.close()

# Test labels go from 0 to 4, which corresponds to classes 1 to 5
test_labels = test_labels + 1 


In [13]:
#Load predictions
with open(os.path.join(base_path, "test_dict.pickle"), "rb") as file:
    test_dict = pickle.load(file,  encoding="latin1")
file.close()

with open(os.path.join(base_path, "predictions.pickle"), "rb") as file:
    predictions = pickle.load(file, encoding="latin1")
file.close()


In [20]:
#Calculate metrics
##RSME
from surprise import accuracy
from scipy.special import softmax

import numpy as np

ss = softmax(predictions[0],axis=1)
pred_y = ss * class_values
pred_y = pred_y.sum(axis=1)
diff =  test_labels - pred_y 
np.sqrt(np.mean(diff**2))



0.9041063586618923

In [21]:
#MAE
from sklearn.metrics import mean_absolute_error
mean_absolute_error(test_labels, pred_y)

0.7082446940253759

In [19]:
#NDCG
from sklearn.metrics import ndcg_score
from scipy import sparse

# Turn predictions into probabilities
ss = softmax(predictions[0],axis=1)

# multiply each class value with the predicted probabilities!
estimated_labels = (ss * class_values).sum(axis=1)

uids = test_dict['test_u_indices']
iids = test_dict['test_v_indices']
r_uis = test_labels
ests = estimated_labels

# Choose a k value!
k_highest_scores = 10

assert(len(uids) == len(iids) == len(r_uis) == len(ests) )    
    
sparse_preds = sparse.coo_matrix( (ests, (uids , iids )) )
sparse_vals = sparse.coo_matrix( (r_uis, (uids , iids )) )

dense_preds = sparse_preds.toarray()
dense_vals = sparse_vals.toarray()

ndcg_score(y_true= dense_vals , y_score= dense_preds, k=k_highest_scores)

0.9103792783561415